<a href="https://colab.research.google.com/github/Rujjul/Synthetic_Dataset_Generator/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Synthetic_Dataset_Generator

In [ ]:
!pip install -q --upgrade bitsandbytes accelerate

In [ ]:
#imports
import os
import requests
from IPython.display import Markdown, display, update_display
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
import torch
import gradio as gr

In [ ]:
#LLM Model

LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
#Quant config

quant_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_compute_dtype=torch.bfloat16,
bnb_4bit_quant_type="nf4"
)

In [ ]:
#Tokenizer

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

In [ ]:
system_message = """
You are a helpful assistant who generates synthetic data based on the user's demand and scenario. The user will provide you with the number of rows, the column name with their respective data types, and some other parameters or constraints if necessary. If you are not able to provide any data regarding something just directly mention that you are unable to do so. Tehn the user will either update or modify their prompt. Do not provide a python script to generate the data. Provide the data as a json with arrays.
"""

user_prompt = f"""
Create a synthetic dataset for an offical document.
Column names and type-
Number of rows: 5
Name: not more than 13 alphabets
Phone Number: 10 digit number
Age: not more than 2 digit number
Occupation: 30 alphabets
PAN ID: 12 alphanumeric characters
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

In [ ]:
response = tokenizer.decode(outputs[0])

In [ ]:
display(Markdown(response))